In [1]:
# coding: utf-8

# Import modules

import pandas as pd
import numpy as np
from numpy.random import permutation
import sys
import nltk
import pickle
import time
from copy import deepcopy

import torch
import torch.nn as nn
import torch.utils.data as utils
import torchwordemb
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import log_loss, roc_auc_score, mean_squared_error

from multiprocessing import Pool

sys.path.append('/home/hugoperrin/Bureau/DataScience/Kaggle/ToxicComment/Models/')
from models import *

sys.path.append('/home/hugoperrin/Bureau/DataScience/Kaggle/ToxicComment/Models/')
from utils import *

# Import data (1D)
train_vect = np.load('/home/hugoperrin/Bureau/Datasets/ToxicComment/Comment2Vec_train.npy')

# Import data (2D)
# train_vect = np.load('/home/hugoperrin/Bureau/Datasets/ToxicComment/Comment2Vec_train_vM.npy')

Xtrain = pd.read_csv('/home/hugoperrin/Bureau/Datasets/ToxicComment/train.csv')
list_classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
targets = Xtrain[list_classes].values

del Xtrain

time1 = time.time()

# Cross validation loop
CV = 3

CV_score = 0

for i in range(CV):

    print('\n---------------------------------------------------\nLoop number {}'.format(i+1))

    random_order = permutation(len(train_vect))

    ## Train test split
    train_comments = train_vect[random_order[:120000]]
    valid_comments = train_vect[random_order[120001:135000]]
    test_comments = train_vect[random_order[135001:]]

    train_labels = train_vect[random_order[:120000]]
    valid_labels = train_vect[random_order[120001:135000]]
    test_labels = train_vect[random_order[135001:]]

    ## Parameters
    use_GPU = True
    batch_size = 512
    num_epoch = 15

    ## Data to tensor
    train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(train_comments), 
                                                   torch.FloatTensor(train_labels))

    valid_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(valid_comments), 
                                                   torch.FloatTensor(valid_labels))

    test_dataset = torch.FloatTensor(test_comments)


    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True, 
                                               num_workers = 8)

    valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                               batch_size=batch_size,
                                               shuffle=False, 
                                               num_workers = 8)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=False, 
                                               num_workers = 8)
    ## Model 
    net = Autoencoder()

    ## Loss function
    criterion = nn.MSELoss()
    
    ## Optimization algorithm
    optimizer = optim.RMSprop(net.parameters(), lr=0.0005, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9)

    ## Training model
    train_autoencoder(num_epoch, net, train_loader, optimizer, criterion, 
                            valid_loader=valid_loader, use_GPU=use_GPU)

    ## Predicting 
    predictions = pd.DataFrame(predict_autoencoder(net, test_loader, use_GPU=use_GPU))
    
    ## Performance measuring
    score = mean_squared_error(test_labels, predictions)

    CV_score += score*(1/CV)

    print("\nModel intermediate score: {}".format(round(score,5)))


print("\nModel final score: {}\n".format(round(CV_score,5)))


time2 = time.time()
diff_time = (time2 - time1)/60
print("Training time is {} minutes\n".format(round(diff_time,1)))


---------------------------------------------------
Loop number 1

>> Learning: 13312 parameters

Epoch: 1, step:   100, training MSE: 0.2434, validation MSE: 0.1684
Epoch: 1, step:   200, training MSE: 0.1658, validation MSE: 0.1559
Epoch: 2, step:   300, training MSE: 0.0983, validation MSE: 0.1475
Epoch: 2, step:   400, training MSE: 0.1479, validation MSE: 0.1424
Epoch: 3, step:   500, training MSE: 0.0414, validation MSE: 0.1371
Epoch: 3, step:   600, training MSE: 0.1412, validation MSE: 0.1332
Epoch: 3, step:   700, training MSE: 0.1381, validation MSE: 0.1325
Epoch: 4, step:   800, training MSE: 0.1290, validation MSE: 0.1310
Epoch: 4, step:   900, training MSE: 0.1351, validation MSE: 0.1306
Epoch: 5, step:  1000, training MSE: 0.0796, validation MSE: 0.1285
Epoch: 5, step:  1100, training MSE: 0.1328, validation MSE: 0.1288
Epoch: 6, step:  1200, training MSE: 0.0316, validation MSE: 0.1251
Epoch: 6, step:  1300, training MSE: 0.1308, validation MSE: 0.1257
Epoch: 6, step:  